In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#IMPORTING NECESSARY PACKAGES
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
items=pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")
print(items)
print(items.columns)

In [ ]:
#Reading necessary input
item_cat=pd.read_csv("../input/competitive-data-science-predict-future-sales/item_categories.csv")
shops=pd.read_csv("../input/competitive-data-science-predict-future-sales/shops.csv")
train=pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
test=pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")

In [ ]:
print(items.head(10))
print(shops.head(10))
print(train.head(10))
print(test.head(10))

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train['date'] = pd.to_datetime(train['date'])
train['date'] = train['date'].apply(lambda dte: dte.strftime("%Y-%m"))

In [ ]:
train.drop(['date_block_num','item_price'] , axis =1, inplace= True)

In [ ]:
df=train.groupby(['date','shop_id','item_id']).sum()
df = train.pivot_table(index=['shop_id','item_id'], columns='date', values='item_cnt_day', fill_value=0)
df.reset_index(inplace=True)
df

In [ ]:
testing = pd.merge(test, df, on=['shop_id','item_id'], how='left')
testing.drop(['ID', '2013-01'], axis=1, inplace=True)

In [ ]:
testing.isna().sum()

In [ ]:
testing=testing.fillna(0)
testing

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline

In [ ]:
y_data = df['2015-10'].values
X_data = df.drop(['2015-10'], axis = 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.2, random_state = 42)

In [ ]:
parameters = {'n_estimators':[50,100,200], 'max_depth':[3,4,5], 'learning_rate':[0.01,0.05,0.1]}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
Results = RandomizedSearchCV(estimator = GradientBoostingRegressor(random_state=42), 
param_distributions = parameters, scoring=None,n_jobs=-1,iid=False, cv=5)
Results.fit(X_train,y_train)

In [ ]:
grad_boost = GradientBoostingRegressor(learning_rate = 0.1, n_estimators = 100, random_state = 42)
grad_boost.fit(X_train, y_train)
print('Train set mse:', mean_squared_error(y_train, grad_boost.predict(X_train)))
print('Test set mse:', mean_squared_error(y_test, grad_boost.predict(X_test)))
print('Train set score:', grad_boost.score(X_train,y_train))

In [ ]:
grad_boost1 = GradientBoostingRegressor(learning_rate = 0.1, n_estimators = 60, random_state = 42)
grad_boost1.fit(X_train, y_train)
print('Train set mse:', mean_squared_error(y_train, grad_boost1.predict(X_train)))
print('Test set mse:', mean_squared_error(y_test, grad_boost1.predict(X_test)))
print('Train set score:', grad_boost1.score(X_train,y_train))

In [ ]:
grad_boost2 = GradientBoostingRegressor(learning_rate = 0.1, n_estimators = 200, random_state = 42)
grad_boost2.fit(X_train, y_train)
print('Train set mse:', mean_squared_error(y_train, grad_boost2.predict(X_train)))
print('Test set mse:', mean_squared_error(y_test, grad_boost2.predict(X_test)))
print('Train set score:', grad_boost2.score(X_train,y_train))

In [ ]:
anslist = grad_boost2.predict(testing)
anslist = list(map(round, anslist))

In [ ]:
answer = pd.read_csv('../input/competitive-data-science-predict-future-sales/sample_submission.csv')
answer.head()

In [ ]:
answer['item_cnt_month'] = anslist
answer.to_csv('answers.csv', index=False)

In [ ]:
##end